# Imports

In [44]:
import pandas as pd
import numpy as np
import sys

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# Load Data 

In [45]:
df_train = pd.read_csv('./datasets/titanic/train.csv')
df_test = pd.read_csv('./datasets/titanic/test.csv')

In [46]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [47]:
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


# Create Predictor and Response Dataframes

In [56]:
X = df_train.copy()
X = X.drop(['PassengerId', 'Name', 'Cabin', 'Ticket'], axis=1)

# Clean Data

In [59]:
X = X.dropna(axis=0)
y = X.Survived
X = X.drop('Survived', axis=1)

# Encode Categorical Features

In [60]:
X = pd.get_dummies(X, columns=['Sex', 'Embarked'])

# Scripts

In [64]:
def eval_models(models, X, y):
    X_train, X_test, y_train, y_test = get_train_test_split(X, y)
    
    ret = []
    print_status_bar(len(models), len(ret))
    
    for name, model in models:
        precision, recall = train_and_get_scores(model, X_train, y_train, X_test, y_test)
        ret.append({'Precision': precision, 'Recall': recall, 'Classifier': name})
        print_status_bar(len(models), len(ret))
    print('\n')
    
    return ret

def get_train_test_split(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    return X_train, X_test, y_train, y_test

def train_and_get_scores(model, X_train, y_train, X_test, y_test, verbose=False):
    classifier = model()
    clf = classifier.fit(X_train, y_train)
    
    y_pred = classifier.predict(X_test)
    # y_score = classifier.decision_function(X_test)

    prec = precision_score(y_test, y_pred)
    recl = recall_score(y_test, y_pred)
    # roc = roc_auc_score(y_test, y_score)
    
    return prec, recl
    
    
def print_status_bar(bar_length, filled_length):
    bar = '=' * filled_length + '-' * (bar_length - filled_length)
    sys.stdout.write('\r[%s]' % bar)
    sys.stdout.flush()

# Models Under Evaluation

In [62]:
models = [('LogisticRegression', LogisticRegression), 
          ('SVM_Classifier', LinearSVC), 
          ('DecisionTreeClassifier', DecisionTreeClassifier),
          ('RandomForestClassifier', RandomForestClassifier),
          ('GradientBosstingClassifier', GradientBoostingClassifier)]

# Test

In [65]:
results_df = pd.DataFrame(eval_models(models, X, y))

[=====]



In [66]:
results_df

,Classifier,Precision,Recall
0,LogisticRegression,0.750000,0.696970
1,SVM_Classifier,0.793103,0.696970
2,DecisionTreeClassifier,0.631579,0.606061
3,RandomForestClassifier,0.747368,0.717172
4,GradientBosstingClassifier,0.766667,0.696970
